<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_P02_GP01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os

# Download dataset using kagglehub
import kagglehub
path = kagglehub.dataset_download("debashishsau/aslamerican-sign-language-aplhabet-dataset")

# List files in the dataset folder
print("Path to dataset files:", path)
files = os.listdir(path)
print("Files in the dataset:", files)

Path to dataset files: /root/.cache/kagglehub/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset/versions/1
Files in the dataset: ['ASL_Alphabet_Dataset']


In [10]:
import os

def list_files_in_directory(directory_path):
    return os.listdir(directory_path)

# Define the subfolder path
subfolder_path = os.path.join(path, 'ASL_Alphabet_Dataset')

# List files in the main dataset folder, training folder, and testing folder
folders = ['asl_alphabet_train', 'asl_alphabet_test']
for folder in folders:
    folder_path = os.path.join(subfolder_path, folder)
    print(f"Files in '{folder}' folder:", list_files_in_directory(folder_path))


Files in 'asl_alphabet_train' folder: ['U', 'I', 'L', 'G', 'P', 'R', 'K', 'J', 'D', 'Y', 'B', 'V', 'Z', 'Q', 'H', 'S', 'T', 'O', 'F', 'W', 'N', 'nothing', 'space', 'C', 'E', 'M', 'X', 'A', 'del']
Files in 'asl_alphabet_test' folder: ['Z_test.jpg', 'V_test.jpg', 'E_test.jpg', 'space_test.jpg', 'K_test.jpg', 'D_test.jpg', 'R_test.jpg', 'J_test.jpg', 'S_test.jpg', 'C_test.jpg', 'P_test.jpg', 'F_test.jpg', 'M_test.jpg', 'N_test.jpg', 'H_test.jpg', 'Q_test.jpg', 'X_test.jpg', 'G_test.jpg', 'I_test.jpg', 'nothing_test.jpg', 'T_test.jpg', 'U_test.jpg', 'L_test.jpg', 'A_test.jpg', 'O_test.jpg', 'W_test.jpg', 'B_test.jpg', 'Y_test.jpg']
